In [41]:
import numpy as np
import pandas as pd

In [37]:
root = '/Users/lukearend/phd/kfx/data/mnt/Ketamine'
sessfiles = pd.read_pickle('/Users/lukearend/phd/kfx/ref/HPC_lfp_paths.pkl')
sessfile = sessfiles[4]
sess = sessfile.split('/')[0]
sess

'2022-08-04_03-45-00_M016_RSK_mPFC_HPC_3_10_30mpk'

In [174]:
lfppath = f'{root}/{sess}/{sess}_g0_imec1/{sess}_g0_t0.imec1.lf.bin'
metapath = lfppath.replace('.bin', '.meta')
with open(metapath, 'r') as f:
    md = {}
    for line in f.read().splitlines():
        key, val = line.split('=')
        if key.startswith('~'):
            key = key[1:]
        md[key] = val

In [167]:
# get the signal sampling rate in Hz
samplerate = float(md['imSampRate'])

In [168]:
# get a number that converts from raw 16-bit memory map values to volts (pre-gain correction)
int2volt = float(md['imAiRangeMax']) / int(md.get('imMaxInt', 512))

In [169]:
# look up the identifier of the original channel on the probe, for each channel we have stored in the data file
saved = md['snsSaveChanSubset']
chanidx = []
for chanset in saved.split(','):
    if len(chanset) == 1:
        chanstr = str(chanset)
    else:
        start, end = chanset.split(':')
        chanidx += list(range(int(start), int(end) + 1))
chanidx = np.array(chanidx)

In [170]:
ap, lf, sy = md['snsApLfSy'].split(',')
chancounts = {'AP': int(ap), 'LF': int(lf), 'SY': int(sy)}
assert chancounts['SY'] == 1
assert chancounts['LF'] == len(chanidx) - 1

In [171]:
probetype = int(md.get('imDatPrb_type', 0))
assert probetype == 0

In [138]:
gains = md['imroTbl'].strip('(').strip(')')
gains = gains.split(')(')[1:]
lfgain = np.array([int(s.split(' ')[4]) for s in gains])
assert np.all(lfgain == lfgain[0])
lfgain = lfgain[0]

In [173]:
nbytes = int(md['fileSizeBytes'])
nchans = int(md['nSavedChans'])
assert nbytes % 2 == 0
assert nbytes % nchans == 0
nsamples = nbytes // 2 // nchans

In [176]:
memmap = np.memmap(lfppath, dtype='int16', mode='r',
                   shape=(nchans, nsamples), offset=0, order='F')

In [179]:
x = memmap[0, :]

In [ ]:
np.array(x)

---

In [ ]:
Lfp, speed, gain, rec, _ , CH_end = load_data(binFullPath, HPC_path_file, PFC_path_file, "HPC", 2)